In [4]:
from abc import ABC, abstractmethod
import socceraction.vaep as vaep
import pandas as pd
import warnings
import os
from statsbombpy import sb

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [13]:
lineups = sb.lineups(match_id=69312)

c:\Users\34626\anaconda3\lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [18]:
lineups

{'Barcelona':     player_id                       player_name  player_nickname  \
 0        3385  Alexis Alejandro Sánchez Sánchez   Alexis Sánchez   
 1        3478         Francesc Fàbregas i Soler    Cesc Fàbregas   
 2        3958   Pedro Eliezer Rodríguez Ledesma            Pedro   
 3        4324             Daniel Alves da Silva       Dani Alves   
 4        5203          Sergio Busquets i Burgos  Sergio Busquets   
 5        5213             Gerard Piqué Bernabéu     Gerard Piqué   
 6        5216              Andrés Iniesta Luján   Andrés Iniesta   
 7        5503    Lionel Andrés Messi Cuccittini     Lionel Messi   
 8       20125          Carles Puyol i Saforcada     Carles Puyol   
 9       20131            Xavier Hernández Creus             Xavi   
 10      20169                      Seydou Kéita     Seydou Kéita   
 11      20176             Víctor Valdés Arribas    Víctor Valdés   
 12      20177         Eric-Sylvain Bilal Abidal      Éric Abidal   
 
     jersey_number 

In [29]:
def extract_player_names_for_team(json_data, team_name):
    player_names_dict = {}

    if team_name in json_data:
        players_df = json_data[team_name]
        for _, player_row in players_df.iterrows():
            player_name = player_row["player_name"]
            player_nickname = player_row["player_nickname"]
            if pd.notna(player_nickname):
                player_names_dict[player_name] = player_nickname

    return player_names_dict

In [30]:
a = extract_player_names_for_team(lineups, 'Espanyol' )

In [19]:
names_dict = {player["player_name"]: player["player_nickname"]
              for team in lineups if "lineup" in team
              for player in team["lineup"]}

## Processing

In [34]:
def read_data(match_id):
    """
    Read StatsBomb eventing data of the selected 'match_id', generating a pandas DataFrame
    with the events and a dictionary of player names and nicknames.
    """
    lineups = sb.lineups(match_id=match_id)
    events = sb.events(match_id=match_id)

    return lineups, events

In [35]:
def extract_player_names_for_team(json_data, team_name):
    player_names_dict = {}

    if team_name in json_data:
        players_df = json_data[team_name]
        for _, player_row in players_df.iterrows():
            player_name = player_row["player_name"]
            player_nickname = player_row["player_nickname"]
            if pd.notna(player_nickname):
                player_names_dict[player_name] = player_nickname

    return player_names_dict

In [49]:
def compute_total_minutes(events):
    """Compute the maximum number of minutes that are used for the passing network.
    The idea is not to have more/less than 11 players in the team because of substitutions or red cards.
    """
    first_red_card_minute = events[events.foul_committed_card.isin(["Second Yellow", "Red Card"])].minute.min()
    first_substitution_minute = events[events.type == "Substitution"].minute.min()
    max_minute = events.minute.max()

    num_minutes = min(first_substitution_minute, first_red_card_minute, max_minute)

    return num_minutes

In [50]:
lineups, df_events = read_data(69312)

c:\Users\34626\anaconda3\lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [47]:
df_events.columns

Index(['bad_behaviour_card', 'ball_receipt_outcome',
       'ball_recovery_recovery_failure', 'block_save_block',
       'carry_end_location', 'clearance_aerial_won', 'clearance_body_part',
       'clearance_head', 'clearance_left_foot', 'clearance_right_foot',
       'counterpress', 'dribble_nutmeg', 'dribble_outcome', 'dribble_overrun',
       'duel_outcome', 'duel_type', 'duration', 'foul_committed_advantage',
       'foul_committed_card', 'foul_committed_type', 'foul_won_advantage',
       'foul_won_defensive', 'goalkeeper_body_part', 'goalkeeper_end_location',
       'goalkeeper_outcome', 'goalkeeper_position', 'goalkeeper_technique',
       'goalkeeper_type', 'id', 'index', 'interception_outcome', 'location',
       'match_id', 'minute', 'miscontrol_aerial_won', 'off_camera', 'out',
       'pass_aerial_won', 'pass_angle', 'pass_assisted_shot_id',
       'pass_body_part', 'pass_cross', 'pass_deflected', 'pass_end_location',
       'pass_goal_assist', 'pass_height', 'pass_inswingin

In [51]:
player_names = extract_player_names_for_team(lineups, "Espanyol")
num_minutes = compute_total_minutes(df_events)